## Music Evaluation

In [1]:
import json
from argparse import ArgumentParser
import midi
import glob
import copy
import os
import numpy as np
import pretty_midi
from pprint import pprint
import pickle
from mgeval.mgeval import core, utils
from sklearn.model_selection import LeaveOneOut

In [2]:
# midi file이 들어있는 dict path를 넣어주세요!

set1dir = '/home/bang/다운로드/maestro-v2.0.0-midi/maestro_midi'
set2dir = '/home/bang/PycharmProjects/POP909-Dataset/pop909_midi'

# set1dir = '/home/bang/PycharmProjects/MusicTransformer-Pytorch/MT_pop_1102_static/classic'
# set2dir = '/home/bang/PycharmProjects/MusicTransformer-Pytorch/MT_pop_1102_static/pop'

In [3]:
set1 = glob.glob(os.path.join(set1dir, '*'))
set2 = glob.glob(os.path.join(set2dir, '*'))

## Number of samples

- set1: classic midi files
- set2: pop midi files

In [4]:
print("set1 size :", len(set1), ", set2 size :", len(set2))

set1 size : 1281 , set2 size : 909


In [5]:
num_samples = min(len(set2), len(set1))

## metrics

- total used pitch
- pitch_range
- avg pitch shift
- avg IOI
- total_pitch class histogram

In [6]:
evalset = { 
            'total_used_pitch': np.zeros((num_samples, 1))
          , 'pitch_range': np.zeros((num_samples, 1))
          , 'avg_pitch_shift': np.zeros((num_samples, 1))
          , 'avg_IOI': np.zeros((num_samples, 1))
          #, 'total_used_note': np.zeros((num_samples, 1))
          # , 'bar_used_pitch': np.zeros((num_samples, args.num_bar, 1))
          # , 'bar_used_note': np.zeros((num_samples, args.num_bar, 1))
          , 'total_pitch_class_histogram': np.zeros((num_samples, 12))
          # , 'bar_pitch_class_histogram': np.zeros((num_samples, args.num_bar, 12))
          #, 'note_length_hist': np.zeros((num_samples, 12))
          #, 'pitch_class_transition_matrix': np.zeros((num_samples, 12, 12))
          #, 'note_length_transition_matrix': np.zeros((num_samples, 12, 12))
          }

In [7]:
metrics_list = list(evalset.keys())

single_arg_metrics = (
    [ 'total_used_pitch'
    , 'avg_IOI'
    , 'total_pitch_class_histogram'
    , 'pitch_range'
    ])

set1_eval = copy.deepcopy(evalset)
set2_eval = copy.deepcopy(evalset)

sets = [ (set1, set1_eval), (set2, set2_eval) ]

In [8]:
def total_pitch_class_histogram_All_instruments(feature):
    """
    total_pitch_class_histogram (Pitch class histogram):
    The pitch class histogram is an octave-independent representation of the pitch content with a dimensionality of 12 for a chromatic scale.
    In our case, it represents to the octave-independent chromatic quantization of the frequency continuum.

    Returns:
    'histogram': histrogram of 12 pitch, with weighted duration shape 12
    """
    
    instrument_num = len(feature['pretty_midi'].instruments)
    piano_roll_list = []
    for i in range(instrument_num):
        piano_roll = feature['pretty_midi'].instruments[i].get_piano_roll(fs=100)
        piano_roll_list.append(piano_roll)
    
    histogram = np.zeros(12)
    for i in range(0, 128):
        pitch_class = i % 12
        for piano_roll_temp in piano_roll_list:
            histogram[pitch_class] += np.sum(piano_roll_temp, axis=1)[i]
    histogram = histogram / sum(histogram)
    return histogram

In [9]:
def pitch_range_All_instruments(feature):
    """
    pitch_range (Pitch range):
    The pitch range is calculated by subtraction of the highest and lowest used pitch in semitones.

    Returns:
    'p_range': a scalar for each sample.
    """
    
    instrument_num = len(feature['pretty_midi'].instruments)
    cnt_pitch_temp = []
    for i in range(instrument_num):
        piano_roll = feature['pretty_midi'].instruments[i].get_piano_roll(fs=100)
        cnt_pitch_temp.append(np.sum(piano_roll, axis=1))

    pitch_index = np.where(np.sum(cnt_pitch_temp, axis=0) > 0)

    p_range = np.max(pitch_index) - np.min(pitch_index)
    

    return p_range

In [13]:
def total_used_note_All_instruments(feature):
    """
    total_used_note (Note count): The number of used notes.
    As opposed to the pitch count, the note count does not contain pitch information but is a rhythm-related feature.

    Args:
    'track_num' : specify the track number in the midi pattern, default is 1 (the second track).

    Returns:
    'used_notes': a scalar for each sample.
    """
    
    pattern = feature['midi_pattern']
    
    temp = []
    for i in range(len(pattern)):
        temp.extend(pattern[i])
    
    pattern = temp
    
    used_notes = 0
    for i in range(0, len(pattern)):
        if type(pattern[i]) == midi.events.NoteOnEvent and pattern[i].data[1] != 0:
            used_notes += 1
    return used_notes

In [14]:
def avg_pitch_shift_All_instruments(feature):
    """
    avg_pitch_shift (Average pitch interval):
    Average value of the interval between two consecutive pitches in semitones.

    Args:
    'track_num' : specify the track number in the midi pattern, default is 1 (the second track).

    Returns:
    'pitch_shift': a scalar for each sample.
    """
    pattern = feature['midi_pattern']
    pattern.make_ticks_abs()
    resolution = pattern.resolution
    total_used_note_value = total_used_note(feature)
    d_note = np.zeros((max(total_used_note_value - 1, 0)))
    # if total_used_note_value == 0:
      # return 0
    # d_note = np.zeros((total_used_note_value - 1))
    
    # 여러 트랙을 하나로 합쳐줌
    pattern = feature['midi_pattern']
    temp = []
    for i in range(len(pattern)):
        temp.extend(pattern[i])
    pattern = temp
    
    current_note = 0
    counter = 0
    for i in range(0, len(pattern)):
        if type(pattern[i]) == midi.events.NoteOnEvent and pattern[i].data[1] != 0:
            if counter != 0:
                d_note[counter - 1] = current_note - pattern[i].data[0]
                current_note = pattern[i].data[0]
                counter += 1
            else:
                current_note = pattern[i].data[0]
                counter += 1
    pitch_shift = np.mean(abs(d_note))
    return pitch_shift

## 평가 진행

In [15]:
# Extract Fetures
for _set, _set_eval in sets:
    for i in range(0, num_samples):
        feature = core.extract_feature(_set[i])
        for metric in metrics_list:
            if metric == 'pitch_range':
                evaluator = pitch_range_All_instruments
            elif metric == 'total_pitch_class_histogram':
                evaluator = total_pitch_class_histogram_All_instruments
            elif metric == 'avg_pitch_shift':
                evaluator = avg_pitch_shift_All_instruments
            else:
                continue
                evaluator = getattr(core.metrics(), metric)
                
            if metric in single_arg_metrics or metric == 'avg_pitch_shift':
                tmp = evaluator(feature)
            # elif metric in bar_metrics:
            #     # print(metric)
            #     tmp = evaluator(feature, 0, args.num_bar)
            # print(tmp.shape)
            else:
                tmp = evaluator(feature, 1)
            _set_eval[metric][i] = tmp

## 평가 결과

In [16]:
set1_TUP = (set1_eval['total_used_pitch']).sum(-1).mean()
set2_TUP = (set2_eval['total_used_pitch']).sum(-1).mean()

print("set1 total used pitch :", set1_TUP)
print("set2 total used pitch :", set2_TUP)

set1 total used pitch : 0.0
set2 total used pitch : 0.0


In [17]:
set1_TCU = (set1_eval['total_pitch_class_histogram']!=0).sum(-1).mean()
set2_TCU = (set2_eval['total_pitch_class_histogram']!=0).sum(-1).mean()

print("set1_TCU :", set1_TCU)
print("set2_TCU :", set2_TCU)

set1_TCU : 11.990099009900991
set2_TCU : 9.4004400440044


In [18]:
set1_PR = (set1_eval['pitch_range']).mean()
set2_PR = (set2_eval['pitch_range']).mean()

print("set1_PR :", set1_PR)
print("set2_PR :", set2_PR)

set1_PR : 68.18921892189219
set2_PR : 53.34653465346535


In [19]:
set1_APS = set1_eval['avg_pitch_shift'].sum(-1).mean()
set2_APS = set2_eval['avg_pitch_shift'].sum(-1).mean()

print("set1_APS :", set1_APS)
print("set2_APS :", set2_APS)

set1_APS : 11.58998431665204
set2_APS : 6.638152930075139


In [20]:
set1_APS = set1_eval['avg_pitch_shift'].sum(-1).mean()
set2_APS = set2_eval['avg_pitch_shift'].sum(-1).mean()

print("set1_APS 2 :", set1_APS)
print("set2_APS 2 :", set2_APS)

set1_APS 2 : 11.58998431665204
set2_APS 2 : 6.638152930075139


In [21]:
set1_IOI = set1_eval['avg_IOI'].sum(-1).mean()
set2_IOI = set2_eval['avg_IOI'].sum(-1).mean()

print("set1_IOI :", set1_IOI)
print("set2_IOI :", set2_IOI)

set1_IOI : 0.0
set2_IOI : 0.0
